In [ ]:
!pip install datasets


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#path to drive folder
%cd /content/drive/MyDrive/capstone/FinQ&A/

/content/drive/MyDrive/capstone/FinQ&A


In [ ]:
#importing libraries
import json
import torch
from datasets import Dataset
import pandas as pd
import re

from transformers import BertForQuestionAnswering, BertTokenizerFast, Trainer, TrainingArguments

In [ ]:
#importing libraries
import json

#Loading the json file
with open('train1.json') as f:
    train_data = json.load(f)

train_data[-1]

{'table': {'uid': '0131320a-1ed4-470e-90f8-7a68e9c8c766',
  'table': [['USDm', '2019', '2018'],
   ['Fair value of derivatives:', '', ''],
   ['Derivative financial instruments regarding freight and bunkers:', '', ''],
   ['Forward freight agreements', '-0.3', '0.5'],
   ['Bunker swaps', '-', '-1.2'],
   ['Derivative financial instruments regarding interest and currency exchange rate:',
    '',
    ''],
   ['Forward exchange contracts', '-0.4', '-1.8'],
   ['Interest rate swaps', '-11.1', '2.8'],
   ['Fair value of derivatives as of 31 December', '-11.8', '0.3']]},
 'paragraphs': [{'uid': '0d321e7b-790b-41bd-afd6-6668c5713c08',
   'order': 1,
   'text': 'NOTE 19 – DERIVATIVE FINANCIAL INSTRUMENTS'},
  {'uid': '43da24be-9b65-4043-acaf-a3a5090729bd',
   'order': 2,
   'text': 'Please refer to note 21 “Financial Instruments” for further information on fair value hierarchies.'}],
 'questions': [{'uid': '69b23654-bdc4-41ad-af90-5e60ffc0461c',
   'order': 1,
   'question': 'What information 

In [ ]:
# eval dataset

with open('test1.json') as f:
    test_data = json.load(f)

test_data[-1]

{'table': {'uid': '8db2d2a5-fdcd-45c2-be2f-813cba4fbca7',
  'table': [['Years Ended December 31,', '', ''],
   ['', '2019', '2018'],
   ['Telecommunication - Maintenance', '$86.8', '$87.0'],
   ['Telecommunication - Installation', '33.2', '41.5'],
   ['Power - Operations, Maintenance & Construction Support', '19.9', '31.0'],
   ['Power - Cable Installation & Repair', '32.6', '34.8'],
   ['Total revenue from contracts with customers', '172.5', '194.3'],
   ['Other revenue', '—', '—'],
   ['Total Marine Services segment revenue', '$172.5', '$194.3']]},
 'paragraphs': [{'uid': 'bff84e8c-0a32-44f3-bcf2-a19f657ad07f',
   'order': 1,
   'text': 'Marine Services Segment GMSL generally generates revenue by providing maintenance services for subsea telecommunications cabling, installing subsea cables, providing installation, maintenance and repair of fiber optic communication and power infrastructure to offshore oil and gas platforms, and installing inter-array power cables for use in offshore 

In [ ]:
# there is elipses like '...' in the question and answers. So fixing the broken words.
text_data = train_data #either train_data or test_data
# Function to clean broken words based on the paragraph text
def clean_broken_words(text, paragraphs):
    # Remove extra dots or ellipses and identify partial words
    ellipsis_pattern = re.compile(r'\b\w*\…\w*\b')
    broken_words = ellipsis_pattern.findall(text)

    if broken_words:
        # Clean the text by removing the ellipses
        cleaned_text = re.sub(r'\…+', '', text)
        question_words = cleaned_text.split()

        # Split words from paragraphs
        for para in paragraphs:
            paragraph_words = para['text'].split()
            for i, word in enumerate(question_words):
                # If word was broken and is not in the paragraph, look for a match
                if word not in paragraph_words:
                    for para_word in paragraph_words:
                        # If part of the word is found in the paragraph word, replace it
                        if word in para_word:
                            question_words[i] = para_word

        # Join the cleaned words back into the sentence
        cleaned_text = ' '.join(question_words)
        return cleaned_text
    return text

# Iterate over questions and answers and clean them
for entry in text_data:
    paragraphs = entry.get('paragraphs', [])

    # Clean each question and its answers
    for question in entry.get('questions', []):
        # Clean the question text
        question['question'] = clean_broken_words(question['question'], paragraphs)

        # Clean the answers
        # question['answer'] = [clean_broken_words(ans, paragraphs) for ans in question['answer']]


# Save the cleaned JSON file
output_file = "cleaned_train_data.json"
with open(output_file, 'w') as outfile:
    json.dump(text_data, outfile, indent=4)

print(f"Cleaned JSON saved to: {output_file}")


Cleaned JSON saved to: cleaned_train_data.json


In [ ]:
# loading the clean json file
with open('cleaned_train_data.json') as f:
    data = json.load(f)

In [ ]:
def table_row_to_text( header, row):
    '''
    Convert a table row to text using the provided header.
    Constructs descriptive sentences from header and row values.
    '''
    res = ""

    # Add the first header item if it exists
    # for index in title:
    #   if index and isinstance(index, str):
    #     res += (index + " ")

    # Iterate through the header and row, skipping the first item
    for head, cell in zip(header[1:], row[1:]):
        if cell:  # Check if the cell is not empty
            res += f"the {row[0]} of {head} is {cell} ; "

    res = remove_space(res)
    return res.strip()


def remove_space(text_in):
    '''Remove extra spaces from the input text.'''
    return " ".join(text_in.split())  # Simplified to remove extra spaces


In [ ]:
data_train_json = []
data_test_json = []
data_json = []

for entry in data:
    context = ""
    paragraph_list = entry.get('paragraphs', [])  # Get the list of paragraphs
    table_dict = entry.get('table', {})  # Get the list of tables
    for para in paragraph_list:
        if isinstance(para, dict):  # Ensure para is a dictionary
            text = para.get('text', '')  # Access 'text' field in each paragraph
            if text:
                context += text + " "


    # Process table if it exists and is structured correctly
    if isinstance(table_dict, dict):
        # Get the table data from the dictionary
        table_data = table_dict.get('table', [])
        if table_data:
            # Assuming the first row is the header and the rest are rows
            header = table_data[1]  # Second row is usually the header
            rows = table_data[2:]  # Data rows start from the third row
            for row in rows:
                text_output = table_row_to_text(header, row)
                context += text_output + " "



    question_list = entry.get('questions', [])  # Get the list of questions
    for item in question_list:
        question_text = item.get('question', "")  # Get the question text
        answer_list = item.get('answer', [])  # Get the list of answers

        # Ensure that answer_list is consistent (as a list)
        if isinstance(answer_list, list):
            answer_list = " | ".join(answer_list)  # Join list items into a single string

            data_json.append({
              'context': context,
              'question':question_text,
              'answer': answer_list,
            })


In [ ]:
data_test_json = data_json

In [ ]:
len(data_test_json)

918

In [ ]:
data_train_json = data_json

In [ ]:
len(data_train_json)

7367

In [ ]:
# run this only for eval_dataset
test_data = data_test_json[0:500]

In [ ]:
# working on 2000 samples
train_data = data_train_json[0:2000]

In [ ]:
# Convert to Hugging Face Dataset
train_df = pd.DataFrame(train_data)
traindataset = Dataset.from_pandas(train_df)

# for eval dataset
test_df = pd.DataFrame(test_data)
testdataset = Dataset.from_pandas(test_df)


In [ ]:
traindataset['question'][:20]

['What does the Weighted average actuarial assumptions consist of?',
 'How much is the 2019 rate of inflation?',
 'How much is the 2018 rate of inflation?',
 'How was internally developed software capitalised?',
 'How much was the closing net book amount for software under development in 2019?',
 'What was the total cost for 2019?',
 'Which year have greater total accumulated amortisation? ',
 'What is net sales from cheese in 2018 and 2019 respectively?',
 'What is net sales from cream and other in 2018 and 2019 respectively?',
 'What is net sales from ProBugs Kefir in 2018 and 2019 respectively?',
 'Why did the Operating revenues decreased from 2018 to 2019? ',
 'Why did the Operating costs decreased from 2018 to 2019?',
 'Why did the gross margin decreased from 2018 to 2019?',
 'What led to increase in the revenue of ADG on sequential basis?',
 'What led to increase in the revenue of AMS on sequential basis?',
 'What led to increase in the revenue of MDG on sequential basis?',
 'How

In [ ]:
# checking the length of all features for both train and test dataset
assert len(traindataset['context']) == len(traindataset['question']) == len(traindataset['answer'])
assert len(testdataset['context']) == len(testdataset['question']) == len(testdataset['answer'])

In [ ]:
train_df.head()

,context,question,answer
0,Actuarial assumptions The Group’s scheme liabi...,What does the Weighted average actuarial assum...,Rate of inflation | Rate of increase in salari...
1,Actuarial assumptions The Group’s scheme liabi...,How much is the 2019 rate of inflation?,2.9
2,Actuarial assumptions The Group’s scheme liabi...,How much is the 2018 rate of inflation?,2.9
3,11 Intangible assets (continued) (a) Intangibl...,How was internally developed software capitali...,at cost less accumulated amortisation.
4,11 Intangible assets (continued) (a) Intangibl...,How much was the closing net book amount for s...,"16,284"


In [ ]:
train_df.shape, test_df.shape

((2000, 3), (500, 3))

In [ ]:
train_df.isnull().sum()

,0
context,0
question,0
answer,0


In [ ]:
# Function to find answer start positions when not provided
def find_answer_positions(context, answer):
    # Debugging: Print the context and answer for visibility
    print(f"Context: {context[:100]}...")  # Print the first 100 characters for brevity
    print(f"Answer: {answer}")

    # Find the position of the answer in the context


    # Check if the answer contains '|', indicating multiple parts
    if '|' in answer:
        answer_parts = [part.strip() for part in answer.split('|')]  # Split and strip each part
        # Find the position of each part in the context
        positions = []
        for part in answer_parts:
            start_idx = context.find(part)
            if start_idx == -1:
                continue
            end_idx = start_idx + len(part)
            positions.append((start_idx, end_idx))
    else:
        start_idx = context.find(answer)  # Treat as a single answer

    if start_idx == -1:
        # raise ValueError(f"Answer '{answer}' not found in context.")
        start_idx = 0
    end_idx = start_idx + len(answer)
    return start_idx, end_idx

In [ ]:
# Function to detect if a question involves a mathematical operation
def detect_math_operation(question):
    # Keywords for different operations
    operations = {
        'sum': ['sum', 'add', 'total'],
        'difference': ['difference', 'subtract', 'minus'],
        'compare': ['compare', 'greater', 'lesser'],
        'max': ['max', 'maximum', 'highest'],
        'min': ['min', 'minimum', 'lowest'],
        'percentage': ['percentage', 'percent', 'growth']
    }

    for operation, keywords in operations.items():
        for keyword in keywords:
            if keyword in question.lower():
                return operation
    return None

In [ ]:
# Preprocessing function for the dataset
def preprocess_data(examples):
    questions = examples['question']  # Extracting questions
    contexts = examples['context']  # Extracting contexts
    answers = examples['answer']  # Extracting answers

    # Debugging: Print lengths of questions and contexts
    if len(questions) != len(contexts):
        raise ValueError(f"Number of questions ({len(questions)}) does not match number of contexts ({len(contexts)})")

    # Tokenization
    inputs = tokenizer(
        questions,
        contexts,
        max_length=512,
        truncation=True,
        padding='max_length',
        return_tensors="pt",
        # return_attention_mask=True,  # Include attention masks
        # return_token_type_ids=True,   # Include token type IDs if necessary
        # return_overflowing_tokens=True,  # Return overflowing tokens if they exist
        # return_offsets_mapping=True,  # Get offset mapping
    )

    start_positions = []
    end_positions = []

    for i in range(len(answers)):
        answer = answers[i]  # Get the answer for the i-th question
        context = contexts[i]  # Get the context for the i-th question

        # Debugging: Print context and answer
        print(f"Processing question {i + 1}/{len(answers)}")

        try:
            # Find start and end positions of the answer in the context
            start_idx, end_idx = find_answer_positions(context, answer)

        except ValueError as e:
            # If the answer is not found, continue to the next example or assign default positions
            print(e)
            start_idx = 0
            end_idx = 0

        # Convert character indices to token indices
        token_start_idx = inputs.char_to_token(i, start_idx)
        token_end_idx = inputs.char_to_token(i, end_idx - 1)  # Adjust end_idx to last character of the answer

        # Handle cases where the token indices are not found
        if token_start_idx is None:
            token_start_idx = tokenizer.model_max_length
        if token_end_idx is None:
            token_end_idx = tokenizer.model_max_length

        start_positions.append(token_start_idx)
        end_positions.append(token_end_idx)

    # Update inputs with start and end positions
    inputs.update({
        'start_positions': start_positions,
        'end_positions': end_positions
    })
    print(f"Input IDs shape: {inputs['input_ids'].shape}")
    print(f"Start positions: {start_positions}")
    print(f"End positions: {end_positions}")

    return inputs


In [ ]:
# Tokenize and process the dataset
train_dataset = traindataset.map(preprocess_data, batched=True, batch_size=16)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Streaming output truncated to the last 5000 lines.
Input IDs shape: torch.Size([16, 512])
Start positions: [512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512]
End positions: [512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512]
Processing question 1/16
Context: Our number of employees is as follows: On August 3, 2018, we implemented a plan to restructure our o...
Answer: We recorded a charge of $381,000 in the third quarter of 2018 relating to this reduction in force, consisting primarily of one-time severance payments and termination benefits.
Processing question 2/16
Context: Our number of employees is as follows: On August 3, 2018, we implemented a plan to restructure our o...
Answer: approximately 30% of the Company’s total pre-restructuring workforce
Processing question 3/16
Context: SHARE INFORMATION On December 31, 2019, the total number of issued common shares of ASMI amounted to...
Answer: Netherlands, ABN AMRO Bank NV | 

In [ ]:
eval_dataset = testdataset.map(preprocess_data, batched=True, batch_size=16)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Processing question 1/16
Context: Sales by Contract Type: Substantially all of our contracts are fixed-price type contracts. Sales inc...
Answer: our allowable incurred costs plus a profit which can be fixed or variable depending on the contract’s fee arrangement up to predetermined funding levels determined by the customer
Processing question 2/16
Context: Sales by Contract Type: Substantially all of our contracts are fixed-price type contracts. Sales inc...
Answer: $1,496.5
Processing question 3/16
Context: Sales by Contract Type: Substantially all of our contracts are fixed-price type contracts. Sales inc...
Answer: fixed-price type | cost-plus type | time-and-material type
Processing question 4/16
Context: Sales by Contract Type: Substantially all of our contracts are fixed-price type contracts. Sales inc...
Answer: 2019
Processing question 5/16
Context: Net sales by segment and industry end market(1) were as follows: (1) Industry end market information...
Answer: consistently with

In [ ]:
# Function to perform mathematical operations
def perform_math_operation(operation, values):
    if operation == 'sum':
        return sum(values)
    elif operation == 'difference':
        return values[0] - values[1]  # Assuming two values
    elif operation == 'max':
        return max(values)
    elif operation == 'min':
        return min(values)
    elif operation == 'percentage':
        return (values[0] - values[1]) / values[1] * 100  # Assuming (new - old) / old * 100
    elif operation == 'average':
        return sum(values) / len(values)
    else:
        raise ValueError(f"Unsupported operation: {operation}")

In [ ]:
# Tokenizer and model initialization
model_name = 'bert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained(model_name)

# Initialize the BERT model for question-answering
model = BertForQuestionAnswering.from_pretrained(model_name)


# Training arguments and setup remain the same
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=4,
    num_train_epochs=20,
    weight_decay=0.01,
    max_grad_norm=1.0,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",                   # Save a model checkpoint at the end of each epoch
    save_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,nan
2,No log,nan
3,No log,nan


TrainOutput(global_step=375, training_loss=2.3532600911458332, metrics={'train_runtime': 564.6646, 'train_samples_per_second': 10.626, 'train_steps_per_second': 0.664, 'total_flos': 1567780540416000.0, 'train_loss': 2.3532600911458332, 'epoch': 3.0})

In [ ]:
# Custom function to perform inference and handle math operations
def answer_question(question, context):
    # Tokenize the question and context
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt')
    input_ids = inputs["input_ids"].tolist()[0]

    # Detect if this is a mathematical question
    math_operation = detect_math_operation(question)

    if math_operation:
        # Perform mathematical operation
        # Extract numeric values from the context
        values = list(map(float, re.findall(r'\d+\.?\d*', context)))
        result = perform_math_operation(math_operation, values)
        return f"The result of {math_operation} is {result}"

    else:
        # Standard QA model flow
        outputs = model(**inputs)
        answer_start = torch.argmax(outputs.start_logits)
        answer_end = torch.argmax(outputs.end_logits) + 1

        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
        return answer